In [1]:
from cnn_method import *
from cnn_preproc_function import *
from multi_cnn_preproc_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os


# optparser = optparse.OptionParser()
# optparser.add_option("-a", "--assetindex", default=0, help="assetindex")
# optparser.add_option("-d", "--gpudevice", default="1", help="gpudevice")
# opts = optparser.parse_args()[0]

assetindex = 0
gpudevice = "1"

# assetindex = int(opts.assetindex)
# gpudevice = opts.gpudevice

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

#36
possible_asset = ['EWZ', 'IWM', 'SPY', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'IYR',
       'XLV', 'OIH', 'EZU', 'XLB', 'EWT', 'XLY', 'EWH', 'EWW', 'DIA', 'EWY',
       'IBB', 'EWC', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'EURGBP',
       'XAGUSD', 'XAUUSD', 'EURUSD', 'EWJ', 'SMH', 'EWG', 'AUDUSD', 'EURJPY',
       'NZDUSD']


if assetindex > len(possible_asset):
    raise Exception("assetindex > len")


Using TensorFlow backend.


In [2]:

print("=============================")
print(possible_asset[assetindex])
print("=============================")
DATA_PARAMS = dict()
DATA_PARAMS["raw_data_file"] = "./DATA/82_ETF_FOREX_1_DAY.csv"
DATA_PARAMS["end_split"] = [datetime.datetime(2011,1,1), datetime.datetime(2013,1,1), datetime.datetime(2015,1,1), datetime.datetime(2018,1,1)]
DATA_PARAMS["TARGET_TO_PREDICT"] = "Multi"
DATA_PARAMS["SELECTED_COLS"] = ["SPY", "EWZ", "SMH", "IBB", "EURUSD", "AUDUSD", "GBPUSD"]
DATA_PARAMS["FUTURE_PERIOD_PREDICT"] = 5
DATA_PARAMS["TARGET_FUNCTION"] = "cumulative_returns"
DATA_PARAMS["SEQ_LEN"] = 60
DATA_PARAMS["TARGET_THRESHOLD"] = 0.001
DATA_PARAMS["FLIP"] = False

MODEL_PARAMS = dict()
MODEL_PARAMS["BATCH_SIZE"] = 64
MODEL_PARAMS["EPOCHS"] = 500
MODEL_PARAMS["PATIENCE"] = 500
# MODEL_PARAMS["LEARNING_RATE"] = 0.005
MODEL_PARAMS["LEARNING_RATE"] = 0.0001
MODEL_PARAMS["monitor_loss"] = "val_loss"
MODEL_PARAMS["mode_loss"] = "min"
INIT_TIME =  str(int(time.time()))
MODEL_PARAMS["outputmain_folder"] = os.path.join("output", "RNN_Multi_1_long_60_5_day")
MODEL_PARAMS["project_folder"] = os.path.join(MODEL_PARAMS["outputmain_folder"], DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME)
MODEL_PARAMS["models_folder"] = os.path.join(MODEL_PARAMS["outputmain_folder"], DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME, "models")
MODEL_PARAMS["logs_folder"] = os.path.join(MODEL_PARAMS["outputmain_folder"], DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME, "logs")

print("Initialize Parameters: Done!")

locals().update(DATA_PARAMS)
locals().update(MODEL_PARAMS)




EWZ
Initialize Parameters: Done!


In [3]:
df = load_data_daily_close_missing(raw_data_file)
#Create target
df, X, Y, x_columns, y_columns = multi_create_target(df, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP, selected_cols = SELECTED_COLS)
#Split df and get index
start_index, end_index = get_index_from_date(df, end_split)
#Scale
X, scaler = scale_x(X, start_index, end_index, scale_method = "standard")
#TSGenerator
train_data_gen, val_data_gen, shape_x, shape_y = TSGenerator0(X, Y, SEQ_LEN, BATCH_SIZE, start_index, end_index)


Load Data: Done!
Clean Data: Done!
Scaling: Done!
(64, 60, 36) (64, 7)
Number of batches per epoch: 37
TSGenerator: Done!


In [4]:

init_dir(models_folder)
init_dir(logs_folder)
print("Initialize directories: Done!")

save_var(DATA_PARAMS, MODEL_PARAMS, scaler, project_folder)

t0 = time.time()

model = multi_rnn_model_conf_1(shape_x, shape_y)
adm = keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False, decay = 1e-6)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])

tensorboard = keras.callbacks.TensorBoard(log_dir=logs_folder)
filepath = "CNN-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}"
checkpoint = keras.callbacks.ModelCheckpoint("{}/{}.model".format(models_folder, filepath),
                                                   monitor=monitor_loss,
                                                   verbose=1,
                                                   save_best_only=False,
                                                   save_weights_only=False,
                                                   mode=mode_loss)
earlystopping = keras.callbacks.EarlyStopping(monitor=monitor_loss,
                                              mode=mode_loss,
                                              min_delta=0,
                                              patience=PATIENCE,
                                              verbose=1)
history = model.fit_generator(generator=train_data_gen,
                              validation_data=val_data_gen,
                              epochs=EPOCHS,
                              callbacks=[tensorboard, checkpoint, earlystopping],
                              verbose=1)

t1 = time.time()
print("Training Done:", t1-t0)

Initialize directories: Done!
Data Saved!
Epoch 1/500
37/37 [==============================] - 2s 44ms/step - loss: 7.3362 - acc: 0.0895 - val_loss: 6.7908 - val_acc: 0.0569

Epoch 00001: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-001-6.7908-0.0569.model
Epoch 2/500
37/37 [==============================] - 0s 9ms/step - loss: 7.0473 - acc: 0.0899 - val_loss: 6.7908 - val_acc: 0.0416

Epoch 00002: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-002-6.7908-0.0416.model
Epoch 3/500
37/37 [==============================] - 0s 10ms/step - loss: 7.1521 - acc: 0.0866 - val_loss: 6.7909 - val_acc: 0.0503

Epoch 00003: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-003-6.7909-0.0503.model
Epoch 4/500
37/37 [==============================] - 0s 10ms/step - loss: 7.0673 - acc: 0.0908 - val_loss: 6.7904 - val_acc: 0.0569

Epoch 00004: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models

Epoch 34/500
37/37 [==============================] - 0s 9ms/step - loss: 7.2575 - acc: 0.1427 - val_loss: 6.7940 - val_acc: 0.1313

Epoch 00034: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-034-6.7940-0.1313.model
Epoch 35/500
37/37 [==============================] - 0s 9ms/step - loss: 7.1630 - acc: 0.1436 - val_loss: 6.7941 - val_acc: 0.1335

Epoch 00035: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-035-6.7941-0.1335.model
Epoch 36/500
37/37 [==============================] - 0s 9ms/step - loss: 7.0599 - acc: 0.1461 - val_loss: 6.7938 - val_acc: 0.1313

Epoch 00036: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-036-6.7938-0.1313.model
Epoch 37/500
37/37 [==============================] - 0s 10ms/step - loss: 7.1936 - acc: 0.1495 - val_loss: 6.7940 - val_acc: 0.1357

Epoch 00037: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-037-6.7940-0.1357.model
Epoch 38/50

37/37 [==============================] - 0s 9ms/step - loss: 7.2580 - acc: 0.1613 - val_loss: 6.7962 - val_acc: 0.1510

Epoch 00067: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-067-6.7962-0.1510.model
Epoch 68/500
37/37 [==============================] - 0s 9ms/step - loss: 7.1918 - acc: 0.1698 - val_loss: 6.7960 - val_acc: 0.1488

Epoch 00068: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-068-6.7960-0.1488.model
Epoch 69/500
37/37 [==============================] - 0s 10ms/step - loss: 7.0989 - acc: 0.1736 - val_loss: 6.7960 - val_acc: 0.1488

Epoch 00069: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-069-6.7960-0.1488.model
Epoch 70/500
37/37 [==============================] - 0s 9ms/step - loss: 7.0234 - acc: 0.1516 - val_loss: 6.7953 - val_acc: 0.1575

Epoch 00070: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-070-6.7953-0.1575.model
Epoch 71/500
37/37 [====

37/37 [==============================] - 0s 12ms/step - loss: 6.9963 - acc: 0.1774 - val_loss: 6.7786 - val_acc: 0.1882

Epoch 00100: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-100-6.7786-0.1882.model
Epoch 101/500
37/37 [==============================] - 0s 10ms/step - loss: 7.0058 - acc: 0.1778 - val_loss: 6.7717 - val_acc: 0.2057

Epoch 00101: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-101-6.7717-0.2057.model
Epoch 102/500
37/37 [==============================] - 0s 10ms/step - loss: 7.0603 - acc: 0.1782 - val_loss: 6.7700 - val_acc: 0.1991

Epoch 00102: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-102-6.7700-0.1991.model
Epoch 103/500
37/37 [==============================] - 0s 9ms/step - loss: 7.0874 - acc: 0.1795 - val_loss: 6.7733 - val_acc: 0.1772

Epoch 00103: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-103-6.7733-0.1772.model
Epoch 104/500
37/37

Epoch 133/500
37/37 [==============================] - 0s 10ms/step - loss: 6.9163 - acc: 0.1677 - val_loss: 6.7576 - val_acc: 0.1838

Epoch 00133: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-133-6.7576-0.1838.model
Epoch 134/500
37/37 [==============================] - 0s 9ms/step - loss: 7.1013 - acc: 0.1647 - val_loss: 6.7571 - val_acc: 0.1751

Epoch 00134: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-134-6.7571-0.1751.model
Epoch 135/500
37/37 [==============================] - 0s 10ms/step - loss: 6.8667 - acc: 0.1689 - val_loss: 6.7673 - val_acc: 0.1860

Epoch 00135: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-135-6.7673-0.1860.model
Epoch 136/500
37/37 [==============================] - 0s 12ms/step - loss: 6.9679 - acc: 0.1579 - val_loss: 6.7740 - val_acc: 0.1947

Epoch 00136: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-136-6.7740-0.1947.model
Epoch

...

37/37 [==============================] - 0s 9ms/step - loss: 6.7454 - acc: 0.1655 - val_loss: 8.1260 - val_acc: 0.1050

Epoch 00364: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-364-8.1260-0.1050.model
Epoch 365/500
37/37 [==============================] - 0s 9ms/step - loss: 6.7979 - acc: 0.1639 - val_loss: 8.1991 - val_acc: 0.1204

Epoch 00365: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-365-8.1991-0.1204.model
Epoch 366/500
37/37 [==============================] - 0s 9ms/step - loss: 6.7451 - acc: 0.1706 - val_loss: 8.1836 - val_acc: 0.1028

Epoch 00366: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-366-8.1836-0.1028.model
Epoch 367/500
37/37 [==============================] - 0s 9ms/step - loss: 6.7928 - acc: 0.1677 - val_loss: 8.1275 - val_acc: 0.1116

Epoch 00367: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-367-8.1275-0.1116.model
Epoch 368/500
37/37 [=

37/37 [==============================] - 0s 9ms/step - loss: 6.9011 - acc: 0.1677 - val_loss: 8.3427 - val_acc: 0.0919

Epoch 00397: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-397-8.3427-0.0919.model
Epoch 398/500
37/37 [==============================] - 0s 9ms/step - loss: 6.8898 - acc: 0.1639 - val_loss: 8.2806 - val_acc: 0.1072

Epoch 00398: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-398-8.2806-0.1072.model
Epoch 399/500
37/37 [==============================] - 0s 10ms/step - loss: 6.8197 - acc: 0.1651 - val_loss: 8.3593 - val_acc: 0.1050

Epoch 00399: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-399-8.3593-0.1050.model
Epoch 400/500
37/37 [==============================] - 0s 9ms/step - loss: 6.9042 - acc: 0.1681 - val_loss: 8.2946 - val_acc: 0.1028

Epoch 00400: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-400-8.2946-0.1028.model
Epoch 401/500
37/37 [

37/37 [==============================] - 0s 9ms/step - loss: 6.6805 - acc: 0.1795 - val_loss: 8.4292 - val_acc: 0.0897

Epoch 00430: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-430-8.4292-0.0897.model
Epoch 431/500
37/37 [==============================] - 0s 10ms/step - loss: 6.7986 - acc: 0.1723 - val_loss: 8.4801 - val_acc: 0.1094

Epoch 00431: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-431-8.4801-0.1094.model
Epoch 432/500
37/37 [==============================] - 0s 9ms/step - loss: 6.8740 - acc: 0.1841 - val_loss: 8.5124 - val_acc: 0.1094

Epoch 00432: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-432-8.5124-0.1094.model
Epoch 433/500
37/37 [==============================] - 0s 12ms/step - loss: 6.8998 - acc: 0.1854 - val_loss: 8.5170 - val_acc: 0.1094

Epoch 00433: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-433-8.5170-0.1094.model
Epoch 434/500
37/37 

Epoch 463/500
37/37 [==============================] - 0s 10ms/step - loss: 6.8530 - acc: 0.2048 - val_loss: 8.5917 - val_acc: 0.1116

Epoch 00463: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-463-8.5917-0.1116.model
Epoch 464/500
37/37 [==============================] - 0s 11ms/step - loss: 6.7293 - acc: 0.1795 - val_loss: 8.4803 - val_acc: 0.1204

Epoch 00464: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-464-8.4803-0.1204.model
Epoch 465/500
37/37 [==============================] - 0s 11ms/step - loss: 6.8005 - acc: 0.1985 - val_loss: 8.5157 - val_acc: 0.1225

Epoch 00465: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-465-8.5157-0.1225.model
Epoch 466/500
37/37 [==============================] - 0s 10ms/step - loss: 6.7156 - acc: 0.1976 - val_loss: 8.6365 - val_acc: 0.1247

Epoch 00466: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-466-8.6365-0.1247.model
Epoc

Epoch 496/500
37/37 [==============================] - 0s 11ms/step - loss: 6.6784 - acc: 0.1981 - val_loss: 8.6476 - val_acc: 0.1182

Epoch 00496: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-496-8.6476-0.1182.model
Epoch 497/500
37/37 [==============================] - 1s 15ms/step - loss: 6.9597 - acc: 0.1883 - val_loss: 8.5960 - val_acc: 0.1138

Epoch 00497: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-497-8.5960-0.1138.model
Epoch 498/500
37/37 [==============================] - 0s 11ms/step - loss: 6.7962 - acc: 0.2010 - val_loss: 8.8169 - val_acc: 0.1247

Epoch 00498: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-498-8.8169-0.1247.model
Epoch 499/500
37/37 [==============================] - 0s 11ms/step - loss: 6.7947 - acc: 0.2145 - val_loss: 8.7038 - val_acc: 0.1050

Epoch 00499: saving model to output/RNN_Multi_1_long_60_5_day/Multi_1540459285/models/CNN-499-8.7038-0.1050.model
Epoc